# Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import os
import shutil
import cv2
import random
from tqdm import tqdm

from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam, RMSprop

from os import getcwd
from absl import logging
logging.set_verbosity(logging.ERROR)

print('\u2022 Using TensorFlow Version:', tf.__version__)
print('\u2022 GPU Device Found.' if tf.config.list_physical_devices('GPU') else '\u2022 GPU Device Not Found. Running on CPU')

• Using TensorFlow Version: 2.10.0
• GPU Device Found.


In [2]:
import shutil
import os

# Nama folder yang ingin dihapus
folders_to_delete = ['data', 'data_resized', 'data_augmented', 'data_cropped', 'data_splitted']

# Loop melalui setiap folder dan hapus jika ada
for folder in folders_to_delete:
    if os.path.exists(folder):
        shutil.rmtree(folder)
        print(f"Folder '{folder}' telah dihapus.")
    else:
        print(f"Folder '{folder}' tidak ditemukan.")

Folder 'data' tidak ditemukan.
Folder 'data_resized' tidak ditemukan.
Folder 'data_augmented' tidak ditemukan.
Folder 'data_cropped' tidak ditemukan.
Folder 'data_splitted' tidak ditemukan.


# Import Dataset

input: `Datasets` <br>

In [3]:
import os

# Base path
base_path = "Datasets"

# Get all folder names in the base_path
folder_names = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]

# Create variables dynamically
for folder in folder_names:
    var_name = f"{folder.lower()}_path"
    var_value = os.path.join(base_path, folder)
    globals()[var_name] = var_value
    print(f"{var_name} = '{var_value}'")

chitato_68_g_path = 'Datasets\Chitato_68_g'
coca_cola_390_ml_path = 'Datasets\Coca_Cola_390_ml'
fanta_390_ml_path = 'Datasets\Fanta_390_ml'
fitbar_20_g_path = 'Datasets\Fitbar_20_g'
frisian_flag_kotak_225ml_path = 'Datasets\Frisian_Flag_kotak_225ml'
garuda_kacang_garing_160_g_path = 'Datasets\Garuda_Kacang_Garing_160_g'
good_day_botol_250_ml_path = 'Datasets\Good_day_botol_250_ml'
hydro_coco_250_ml_path = 'Datasets\Hydro_Coco_250_ml'
indomiee_goreng_85_g_path = 'Datasets\Indomiee_Goreng_85_g'
kratingdeng_150_ml_path = 'Datasets\Kratingdeng_150_ml'
larutan_cap_kaki_tiga_botol_200_ml_path = 'Datasets\Larutan_Cap_Kaki_Tiga_Botol_200_ml'
larutan_cap_kaki_tiga_kaleng_path = 'Datasets\Larutan_cap_kaki_tiga_Kaleng'
milo_240_ml_path = 'Datasets\Milo_240_ml'
mizone_500_ml_path = 'Datasets\Mizone_500_ml'
nabati_50_g_path = 'Datasets\Nabati_50_g'
nestcafe_kaleng_240_ml_path = 'Datasets\Nestcafe_kaleng_240_ml'
nutriboost_300_ml_path = 'Datasets\Nutriboost_300_ml'
oreo_123.5_g_path = 'Datasets\Oreo

# Preprocessing

## Splitting Dataset

input: `Datasets` <br>
output: `data_splitted`

v1

In [4]:
# Definisikan direktori sumber dan tujuan
source_dir = 'Datasets'
destination_dir = 'data_splitted'
base_dir = destination_dir
os.makedirs(base_dir, exist_ok=True)

# Mendapatkan nama kategori dari direktori sumber
folder_names = os.listdir(source_dir)

# Buat direktori train, val, dan test
for split in ['train', 'val', 'test']:
    for folder in folder_names:
        os.makedirs(os.path.join(base_dir, split, folder), exist_ok=True)

# Fungsi untuk memisahkan dan menyalin file
def split_and_copy_files(category_path, category_name):
    files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    
    # Split data menjadi train, val, dan test
    train_files, temp_files = train_test_split(files, test_size=0.8, random_state=42)
    val_files, test_files = train_test_split(temp_files, test_size=0.5, random_state=42)
    
    # Salin file ke direktori train
    for file in train_files:
        shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'train', category_name, file))
    
    # Salin file ke direktori val
    for file in val_files:
        shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'val', category_name, file))
    
    # Salin file ke direktori test
    for file in test_files:
        shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'test', category_name, file))

# Memisahkan dan menyalin file ke direktori train, val, dan test
for folder in folder_names:
    category_path = os.path.join(source_dir, folder)
    category_name = folder
    split_and_copy_files(category_path, category_name)

v2

In [5]:
# # Definisikan direktori sumber dan tujuan
# source_dir = "data_augmented"
# destination_dir = "data"
# base_dir = destination_dir

# # Buat direktori train dan val
# os.makedirs(base_dir, exist_ok=True)
# folder_names = os.listdir(source_dir)  # Mendapatkan nama kategori dari direktori sumber

# for split in ['train', 'val']:
#     for folder in folder_names:
#         os.makedirs(os.path.join(base_dir, split, folder), exist_ok=True)

# # Fungsi untuk memisahkan dan menyalin file
# def split_and_copy_files(category_path, category_name):
#     files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    
#     # Split data menjadi train dan val
#     train_files, val_files = train_test_split(files, test_size=0.2, random_state=42)
    
#     # Salin file ke direktori train
#     for file in train_files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'train', category_name, file))
    
#     # Salin file ke direktori val
#     for file in val_files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'val', category_name, file))

# # Memisahkan dan menyalin file ke direktori train dan val
# for folder in folder_names:
#     category_path = os.path.join(source_dir, folder)
#     category_name = folder
#     split_and_copy_files(category_path, category_name)


In [6]:
# # Definisikan direktori sumber dan tujuan
# source_dir = "Datasets/test/"
# destination_dir = "data"
# base_dir = destination_dir

# # Buat direktori test
# os.makedirs(base_dir, exist_ok=True)
# folder_names = os.listdir(source_dir)  # Mendapatkan nama kategori dari direktori sumber

# for split in ['test']:
#     for folder in folder_names:
#         os.makedirs(os.path.join(base_dir, split, folder), exist_ok=True)

# # Fungsi untuk menyalin file ke direktori test
# def copy_files_to_test(category_path, category_name):
#     files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    
#     # Salin file ke direktori test
#     for file in files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'test', category_name, file))

# # Menyalin file ke direktori test
# for folder in folder_names:
#     category_path = os.path.join(source_dir, folder)
#     category_name = folder
#     copy_files_to_test(category_path, category_name)

v3

In [7]:
# # Definisikan direktori sumber dan tujuan
# source_dir = "data_augmented"
# destination_dir = "data"
# base_dir = destination_dir

# # Buat direktori train
# os.makedirs(base_dir, exist_ok=True)
# folder_names = os.listdir(source_dir)  # Mendapatkan nama kategori dari direktori sumber

# for folder in folder_names:
#     os.makedirs(os.path.join(base_dir, 'train', folder), exist_ok=True)

# # Fungsi untuk memisahkan dan menyalin file
# def split_and_copy_files(category_path, category_name):
#     files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    
#     # Split data menjadi train saja (tidak ada val)
#     train_files = files  # Semua file dianggap sebagai train
    
#     # Salin file ke direktori train
#     for file in train_files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'train', category_name, file))

# # Memisahkan dan menyalin file ke direktori train
# for folder in folder_names:
#     category_path = os.path.join(source_dir, folder)
#     category_name = folder
#     split_and_copy_files(category_path, category_name)

In [8]:
# # Definisikan direktori sumber dan tujuan
# source_dir = "Datasets/test/"
# destination_dir = "data"
# base_dir = destination_dir

# # Buat direktori val dan test
# os.makedirs(base_dir, exist_ok=True)
# folder_names = os.listdir(source_dir)  # Mendapatkan nama kategori dari direktori sumber

# for split in ['val', 'test']:
#     for folder in folder_names:
#         os.makedirs(os.path.join(base_dir, split, folder), exist_ok=True)

# # Fungsi untuk menyalin file ke direktori val dan test
# def split_and_copy_files(category_path, category_name):
#     files = [f for f in os.listdir(category_path) if os.path.isfile(os.path.join(category_path, f))]
    
#     # Split data menjadi val dan test
#     val_files, test_files = train_test_split(files, test_size=0.5, random_state=42)
    
#     # Salin file ke direktori val
#     for file in val_files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'val', category_name, file))
    
#     # Salin file ke direktori test
#     for file in test_files:
#         shutil.copy(os.path.join(category_path, file), os.path.join(base_dir, 'test', category_name, file))

# # Menyalin file ke direktori val dan test
# for folder in folder_names:
#     category_path = os.path.join(source_dir, folder)
#     category_name = folder
#     split_and_copy_files(category_path, category_name)

## Bounding box

input: `data_splitted/train` <br>
output: `data_cropped`

In [9]:
# Fungsi untuk mendeteksi produk dalam gambar
def detect_product(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 128, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour = max(contours, key=cv2.contourArea)
    return contour

# Fungsi untuk menggambar bounding box pada gambar
def draw_bounding_box(image, contour):
    x, y, w, h = cv2.boundingRect(contour)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    return image, (x, y, w, h)

# Fungsi untuk memotong gambar berdasarkan bounding box
def crop_image(image, bbox):
    x, y, w, h = bbox
    cropped_image = image[y:y+h, x:x+w]
    return cropped_image

# Function to process images in a folder and save cropped images to a destination folder
def process_and_crop_images(source_folder_path, destination_folder_path):
    os.makedirs(destination_folder_path, exist_ok=True)
    
    for filename in os.listdir(source_folder_path):
        if filename.endswith(('.jpg', '.jpeg', '.png', '.webp')):
            image_path = os.path.join(source_folder_path, filename)
            img = cv2.imread(image_path)
            
            if img is not None:
                # Deteksi produk dan gambar bounding box
                contour = detect_product(img)
                image_with_box, bbox = draw_bounding_box(img, contour)
                
                # Crop gambar berdasarkan bounding box
                cropped_image = crop_image(img, bbox)
                
                # Save the cropped image to the destination folder
                destination_image_path = os.path.join(destination_folder_path, filename)
                cv2.imwrite(destination_image_path, cropped_image)
                print(f"Cropped and saved {filename} to {destination_image_path}")

In [10]:
# Folder paths
data_resized_path = 'data_splitted/train'
data_cropped_path = 'data_cropped'

# Create the destination folder if it doesn't exist
os.makedirs(data_cropped_path, exist_ok=True)

# Iterate over each product folder in the data_resized_path
for product_folder in os.listdir(data_resized_path):
    source_folder = os.path.join(data_resized_path, product_folder)
    destination_folder = os.path.join(data_cropped_path, product_folder)
    
    if os.path.isdir(source_folder):
        process_and_crop_images(source_folder, destination_folder)

Cropped and saved Bing_0002.jpeg to data_cropped\Chitato_68_g\Bing_0002.jpeg
Cropped and saved Bing_0014.jpeg to data_cropped\Chitato_68_g\Bing_0014.jpeg
Cropped and saved Bing_0021.png to data_cropped\Chitato_68_g\Bing_0021.png
Cropped and saved Bing_0022.webp to data_cropped\Chitato_68_g\Bing_0022.webp
Cropped and saved Bing_0037.webp to data_cropped\Chitato_68_g\Bing_0037.webp
Cropped and saved Bing_0047.webp to data_cropped\Chitato_68_g\Bing_0047.webp
Cropped and saved Bing_0053.jpeg to data_cropped\Chitato_68_g\Bing_0053.jpeg
Cropped and saved Bing_0054.jpeg to data_cropped\Chitato_68_g\Bing_0054.jpeg
Cropped and saved Bing_0055.webp to data_cropped\Chitato_68_g\Bing_0055.webp
Cropped and saved Bing_0057.webp to data_cropped\Chitato_68_g\Bing_0057.webp
Cropped and saved Bing_0058.webp to data_cropped\Chitato_68_g\Bing_0058.webp
Cropped and saved Bing_0059.jpeg to data_cropped\Chitato_68_g\Bing_0059.jpeg
Cropped and saved Bing_0064.webp to data_cropped\Chitato_68_g\Bing_0064.webp
C

## Resize Image

input: `data_cropped` <br>
output: `data_resized`

In [11]:
base_path = "data_cropped"

# Function to resize images in a folder and save to a destination folder
def resize_and_save_images(source_folder_path, destination_folder_path, target_size):
    # Create the destination folder if it doesn't exist
    os.makedirs(destination_folder_path, exist_ok=True)
    
    # Iterate through all files in the source folder
    for filename in os.listdir(source_folder_path):
        # Check if file is an image file
        if filename.endswith(('.jpg', '.jpeg', '.png', '.webp')):
            # Read the image
            image_path = os.path.join(source_folder_path, filename)
            img = cv2.imread(image_path)
            
            # Resize the image
            resized_img = cv2.resize(img, target_size)
            
            # Save the resized image to the destination folder
            destination_image_path = os.path.join(destination_folder_path, filename)
            cv2.imwrite(destination_image_path, resized_img)
            print(f"Resized and saved {filename} to {destination_image_path}")

In [12]:
# Define the target size for resizing images
target_size = (224, 224)

# Loop through each folder and resize images
for folder in folder_names:
    source_folder_path = os.path.join(base_path, folder)
    destination_folder_path = os.path.join("data_resized", folder)
    resize_and_save_images(source_folder_path, destination_folder_path, target_size)

Resized and saved Bing_0002.jpeg to data_resized\Chitato_68_g\Bing_0002.jpeg
Resized and saved Bing_0014.jpeg to data_resized\Chitato_68_g\Bing_0014.jpeg
Resized and saved Bing_0021.png to data_resized\Chitato_68_g\Bing_0021.png
Resized and saved Bing_0022.webp to data_resized\Chitato_68_g\Bing_0022.webp
Resized and saved Bing_0037.webp to data_resized\Chitato_68_g\Bing_0037.webp
Resized and saved Bing_0047.webp to data_resized\Chitato_68_g\Bing_0047.webp
Resized and saved Bing_0053.jpeg to data_resized\Chitato_68_g\Bing_0053.jpeg
Resized and saved Bing_0054.jpeg to data_resized\Chitato_68_g\Bing_0054.jpeg
Resized and saved Bing_0055.webp to data_resized\Chitato_68_g\Bing_0055.webp
Resized and saved Bing_0057.webp to data_resized\Chitato_68_g\Bing_0057.webp
Resized and saved Bing_0058.webp to data_resized\Chitato_68_g\Bing_0058.webp
Resized and saved Bing_0059.jpeg to data_resized\Chitato_68_g\Bing_0059.jpeg
Resized and saved Bing_0064.webp to data_resized\Chitato_68_g\Bing_0064.webp
R

## Image Augmentation

input: `data_resized` <br>
output: `data_augmented`

In [13]:
# Definisikan direktori input dan output
input_dir = 'data_resized'
output_dir = 'data_augmented'
target_count = 500

In [14]:
# Buat direktori output jika belum ada
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Fungsi untuk melakukan augmentasi gambar
def augment_image(image):
    augmented_images = []

    # Rotasi
    for angle in [30, 45, 60, 90, 180, 270]:
        M = cv2.getRotationMatrix2D((image.shape[1]//2, image.shape[0]//2), angle, 1)
        rotated = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
        augmented_images.append(rotated)

    # Flipping
    flipped_h = cv2.flip(image, 1) # Horizontal
    augmented_images.append(flipped_h)
    flipped_v = cv2.flip(image, 0) # Vertical
    augmented_images.append(flipped_v)

    # Scaling
    scale_factors = [0.75, 1.25]
    for factor in scale_factors:
        resized = cv2.resize(image, (0, 0), fx=factor, fy=factor)
        if factor > 1:
            resized = resized[:image.shape[0], :image.shape[1]]
        else:
            pad_x = (image.shape[1] - resized.shape[1]) // 2
            pad_y = (image.shape[0] - resized.shape[0]) // 2
            resized = cv2.copyMakeBorder(resized, pad_y, pad_y, pad_x, pad_x, cv2.BORDER_CONSTANT, value=[0, 0, 0])
        augmented_images.append(resized)

    return augmented_images

# Iterasi melalui setiap folder (kategori)
for category in os.listdir(input_dir):
    category_path = os.path.join(input_dir, category)
    output_category_path = os.path.join(output_dir, category)

    if not os.path.exists(output_category_path):
        os.makedirs(output_category_path)

    images = [os.path.join(category_path, img) for img in os.listdir(category_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
    augmented_count = 0

    # Augmentasi gambar
    for img_path in tqdm(images, desc=f'Processing {category}'):
        img = cv2.imread(img_path)
        augmented_images = augment_image(img)
        
        for aug_img in augmented_images:
            if augmented_count >= target_count:
                break
            output_image_path = os.path.join(output_category_path, f'aug_{augmented_count}.jpg')
            cv2.imwrite(output_image_path, aug_img)
            augmented_count += 1

        if augmented_count >= target_count:
            break

    # Jika masih kurang dari target_count, tambahkan gambar asli
    for img_path in images:
        if augmented_count >= target_count:
            break
        img = cv2.imread(img_path)
        output_image_path = os.path.join(output_category_path, f'orig_{augmented_count}.jpg')
        cv2.imwrite(output_image_path, img)
        augmented_count += 1

print("Data augmentation selesai.")

Processing You_C1000_Water_Orange_500_ml: 100%|██████████| 30/30 [00:00<00:00, 46.73it/s]

Data augmentation selesai.


## Copy Data

In [15]:
def copy_all_files_and_folders(source_folder, destination_folder):
    # Membuat destination_folder jika belum ada
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    
    # Iterasi melalui semua item di source_folder
    for item in os.listdir(source_folder):
        source_path = os.path.join(source_folder, item)
        destination_path = os.path.join(destination_folder, item)
        
        if os.path.isdir(source_path):
            # Jika item adalah direktori, salin seluruh direktori secara rekursif
            if not os.path.exists(destination_path):
                shutil.copytree(source_path, destination_path)
                print(f'Copied directory: {source_path} to {destination_path}')
            else:
                # Jika direktori tujuan sudah ada, kita harus menyalin isinya secara manual
                copy_all_files_and_folders(source_path, destination_path)
        else:
            # Jika item adalah file, salin file tersebut
            try:
                shutil.copy2(source_path, destination_path)
                print(f'Copied file: {source_path} to {destination_path}')
            except PermissionError as e:
                print(f'Permission denied: {source_path}, skipping. Error: {e}')
            except Exception as e:
                print(f'Failed to copy {source_path}. Error: {e}')

# Contoh penggunaan
source_folder_1 = 'data_augmented'
source_folder_2 = 'data_splitted/val'
source_folder_3 = 'data_splitted/test'
destination_folder_1 = 'data/train'
destination_folder_2 = 'data/val'
destination_folder_3 = 'data/test'

copy_all_files_and_folders(source_folder_1, destination_folder_1)
copy_all_files_and_folders(source_folder_2, destination_folder_2)
copy_all_files_and_folders(source_folder_3, destination_folder_3)

Copied directory: data_augmented\Chitato_68_g to data/train\Chitato_68_g
Copied directory: data_augmented\Coca_Cola_390_ml to data/train\Coca_Cola_390_ml
Copied directory: data_augmented\Fanta_390_ml to data/train\Fanta_390_ml
Copied directory: data_augmented\Fitbar_20_g to data/train\Fitbar_20_g
Copied directory: data_augmented\Frisian_Flag_kotak_225ml to data/train\Frisian_Flag_kotak_225ml
Copied directory: data_augmented\Garuda_Kacang_Garing_160_g to data/train\Garuda_Kacang_Garing_160_g
Copied directory: data_augmented\Good_day_botol_250_ml to data/train\Good_day_botol_250_ml
Copied directory: data_augmented\Hydro_Coco_250_ml to data/train\Hydro_Coco_250_ml
Copied directory: data_augmented\Indomiee_Goreng_85_g to data/train\Indomiee_Goreng_85_g
Copied directory: data_augmented\Kratingdeng_150_ml to data/train\Kratingdeng_150_ml
Copied directory: data_augmented\Larutan_Cap_Kaki_Tiga_Botol_200_ml to data/train\Larutan_Cap_Kaki_Tiga_Botol_200_ml
Copied directory: data_augmented\Laruta

## Image Augmentation (v2)

In [16]:
target_size = (224,224)

# Data Directory
data_dir = "data"

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale = 1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        data_dir + "/train",  # This is the source directory for training images
        target_size=target_size,  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='sparse')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        data_dir + "/val",
        target_size=target_size,
        batch_size=10,
        class_mode='sparse',
        shuffle=False
        )

test_generator = test_datagen.flow_from_directory(
        data_dir + "/test",
        target_size=target_size,
        batch_size=10,
        class_mode='sparse',
        shuffle=False
        )

Found 8931 images belonging to 28 classes.
Found 1638 images belonging to 28 classes.
Found 1641 images belonging to 28 classes.
